In [261]:
import sys
import numpy as np
import pandas as pd
import csv
import tensorflow as tf

In [310]:
train_set = pd.read_csv('train.csv', encoding='big5')
row_count = train_set.shape[0]
vectors = []
for i in range(18):
    vectors.append(train_set.iloc[i])
for i in range(18, row_count):
    vectors[i % 18] = pd.concat([vectors[i % 18], train_set.iloc[i]])

series_dict = {}
index = 0
for row in vectors:
    series_dict[index] = row
    index += 1
extracted_train_set = pd.DataFrame(series_dict)
extracted_train_set = extracted_train_set.T
# extracted_train_set = extracted_train_set.drop(['日期', '測站', '測項'], axis=1)
extracted_train_set = extracted_train_set.replace('NR', 0.0)
# extracted_train_set.to_csv('extracted_train.csv')
list_x = []
list_y = []
i = 0
j = 0
while i < extracted_train_set.shape[1]:
    mouth_set = extracted_train_set.iloc[:, i:27 * 20 * (j + 1)]
    mouth_set = mouth_set.drop(['日期', '測站', '測項'], axis=1)
    array = np.array(mouth_set).astype(np.float32)
    for k in range(array.shape[1]):
        if k + 9 >= array.shape[1]:
            break
        mat = array[:, k:k + 9]
        label = array[9, k + 9]
        list_x.append(mat.ravel())
        list_y.append(label)
    i += 27 * 20
    j += 1
train_x = np.array(list_x)
train_y = np.array(list_y)

In [311]:
train_x[0]

array([1.40e+01, 1.40e+01, 1.40e+01, 1.30e+01, 1.20e+01, 1.20e+01,
       1.20e+01, 1.20e+01, 1.50e+01, 1.80e+00, 1.80e+00, 1.80e+00,
       1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00,
       5.10e-01, 4.10e-01, 3.90e-01, 3.70e-01, 3.50e-01, 3.00e-01,
       3.70e-01, 4.70e-01, 7.80e-01, 2.00e-01, 1.50e-01, 1.30e-01,
       1.20e-01, 1.10e-01, 6.00e-02, 1.00e-01, 1.30e-01, 2.60e-01,
       9.00e-01, 6.00e-01, 5.00e-01, 1.70e+00, 1.80e+00, 1.50e+00,
       1.90e+00, 2.20e+00, 6.60e+00, 1.60e+01, 9.20e+00, 8.20e+00,
       6.90e+00, 6.80e+00, 3.80e+00, 6.90e+00, 7.80e+00, 1.50e+01,
       1.70e+01, 9.80e+00, 8.70e+00, 8.60e+00, 8.50e+00, 5.30e+00,
       8.80e+00, 9.90e+00, 2.20e+01, 1.60e+01, 3.00e+01, 2.70e+01,
       2.30e+01, 2.40e+01, 2.80e+01, 2.40e+01, 2.20e+01, 2.10e+01,
       5.60e+01, 5.00e+01, 4.80e+01, 3.50e+01, 2.50e+01, 1.20e+01,
       4.00e+00, 2.00e+00, 1.10e+01, 2.60e+01, 3.90e+01, 3.60e+01,
       3.50e+01, 3.10e+01, 2.80e+01, 2.50e+01, 2.00e+01, 1.90e

In [316]:
# x = tf.placeholder(tf.float32, [162])
w = tf.Variable(tf.zeros([162]))
b = tf.Variable(tf.zeros([1]))

# y = b + np.dot(w, x)

# lost = tf.reduce_mean(tf.square(y_ - y))

def get_y_predict(train_x):
    y = []
    for i in range(train_x.shape[0]):
        x = tf.Variable(train_x[i])
        xs = tf.reshape(x, [162,1])
        ws = tf.reshape(w, [162,1])
        y.append(b + tf.matmul(ws, xs, transpose_a=True))
    return tf.Variable(y)

def get_loss(y_true, y_predict):
    return tf.reduce_mean(tf.reduce_sum((tf.square(y_true - y_predict))))

def sgd_op(loss, learning_rate):
    optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate)
    train_op = optimizer.minimize(loss)
    return train_op

def train():
    y_true = tf.Variable(train_y)
    y_predict = get_y_predict(train_x)
    loss = get_loss(y_true, y_predict)
    train_op = sgd_op(loss, 0.00001)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        for i in range(20000):
            sess.run([train_op])
            
train()

RuntimeError: `loss` passed to Optimizer.compute_gradients should be a function when eager execution is enabled.

In [305]:
w

<tf.Variable 'Variable:0' shape=(162,) dtype=float32, numpy=
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [306]:
x = tf.Variable(train_x[0].astype(float))

In [307]:
x

<tf.Variable 'Variable:0' shape=(162,) dtype=float64, numpy=
array([1.40e+01, 1.40e+01, 1.40e+01, 1.30e+01, 1.20e+01, 1.20e+01,
       1.20e+01, 1.20e+01, 1.50e+01, 1.80e+00, 1.80e+00, 1.80e+00,
       1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00, 1.80e+00,
       5.10e-01, 4.10e-01, 3.90e-01, 3.70e-01, 3.50e-01, 3.00e-01,
       3.70e-01, 4.70e-01, 7.80e-01, 2.00e-01, 1.50e-01, 1.30e-01,
       1.20e-01, 1.10e-01, 6.00e-02, 1.00e-01, 1.30e-01, 2.60e-01,
       9.00e-01, 6.00e-01, 5.00e-01, 1.70e+00, 1.80e+00, 1.50e+00,
       1.90e+00, 2.20e+00, 6.60e+00, 1.60e+01, 9.20e+00, 8.20e+00,
       6.90e+00, 6.80e+00, 3.80e+00, 6.90e+00, 7.80e+00, 1.50e+01,
       1.70e+01, 9.80e+00, 8.70e+00, 8.60e+00, 8.50e+00, 5.30e+00,
       8.80e+00, 9.90e+00, 2.20e+01, 1.60e+01, 3.00e+01, 2.70e+01,
       2.30e+01, 2.40e+01, 2.80e+01, 2.40e+01, 2.20e+01, 2.10e+01,
       5.60e+01, 5.00e+01, 4.80e+01, 3.50e+01, 2.50e+01, 1.20e+01,
       4.00e+00, 2.00e+00, 1.10e+01, 2.60e+01, 3.90e+01, 3.60e+01,
 